OBU XGBoost

Update
1. 分析使用一年份資料，僅保留客戶最新資訊欄位
2. 與兆豐討論後移除變數如下
    - REPORT_FG
    - CDD_STATUS
    - RISK_LEVEL

In [ ]:
# 輸入Module
import os.path
import pandas as pd
import numpy as np
from datetime import date

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn import metrics
import seaborn as sn
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
%matplotlib inline
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import ShuffleSplit
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve #P-R Curve
from sklearn.model_selection import GridSearchCV #Hyperparameters

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold

import xgboost as xgb
import hyperopt
from hyperopt import hp

#### 1. Data Import & Cleaning

In [ ]:
# 資料輸入

# EY補充
# 此處路徑需更新為最新資料存放路徑

OBU = pd.read_csv(r"/home/bigdata93601/DB Result/OBU_result_one_year.csv", dtype=str)
OBU['CYC_MN'] = pd.to_datetime(OBU['CYC_MN'])

In [ ]:
# 刪除FINAL_CRR為空值的row
OBU = OBU[OBU['FINAL_CRR'].notna()]
OBU.reset_index(inplace=True)

# CUST_DUP_NO 空值補0
OBU['CUST_DUP_NO'] = OBU['CUST_DUP_NO'].fillna('0')

# SAR: 非 Y即 N
OBU['SAR'] = np.where(OBU['SAR'] == 'Y', 'Y', 'N')

##### Drop Duplicated data (New Version)
目的: 僅保留在資訊層面最新之帳戶

In [ ]:
OBU = OBU.sort_values('CYC_MN', ascending=False).drop_duplicates(subset=['CUST_ID', 'CUST_DUP_NO'], keep='first')
OBU.reset_index(inplace=True, drop=True)

In [ ]:
OBU['CYC_MN'].value_counts(dropna=False).sort_index()

#### 2. 變數轉換

##### 2.1 類別型變數  

In [ ]:
# EY補充
# 新增欄位的情況下，將類別型變數轉換為Y/N的資料格式。
# 下面範例將以df代表輸入資料，NEW_COLUMN代表新增欄位
# 
# 範例一
# 新增欄位的資料內容為["V", Nan, "V", "V", Nan, ... , "V"]；欲將V轉換為Y，空值(Nan)轉換為N。
# df["NEW_COLUMN"] = np.where(df["NEW_COLUMN"] == "V", "Y", "N")
#
# 範例二
# 新增欄位的資料內容為["Y", "DELAY", "Y", Nan, "Y", ... , "N"]；欲將非Y/N轉換為N，其餘資料維持原狀。
# df["NEW_COLUMN"] = np.where(df["NEW_COLUMN"].isin(["Y", "N"], df["NEW_COLUMN"], "N"))
#
# 範例三
# 新增欄位的資料內容為["01", "02", "03", "04", ..., "01"]；欲將01-02轉換為Y，03-04轉換為N。
# df["NEW_COLUMN"] = df["NEW_COLUMN"].map({"01": "Y", "02": "Y", "03": "N", "04": "N"})


# 需先經過轉換的變數
# 1. CMFCUS1_AML_BUSINESS
def apply_CMFCUS1_AML_BUSINESS(series):
    input = series['CMFCUS1_AML_BUSINESS']
    if (input in ['01', '02', '03', '04', '05', '06', '07', '08']):
        return 'Y'
    else:
        return 'N'

OBU['CMFCUS1_AML_BUSINESS'] = OBU.apply(apply_CMFCUS1_AML_BUSINESS, axis=1)


for i in ['DP_FG', 'LN_FG', 'IX_FG', 'BD_FG', 'FD_FG', 'WM_FG', 'TR_FG', 'EB_FG', 'CD_FG', 'OT_FG']:
    OBU[i] = np.where(OBU[i] == 'V', 'Y', 'N')

    

# 2. CMFCUS25_AE_TYPE
# Turn Na into 'N'; other into 'Y'
OBU['CMFCUS25_AE_TYPE'] = np.where((OBU["CMFCUS25_AE_TYPE"].isin(['1', '2', '3', '4', '5', '6', '7'])), 'Y', 'N')



# 全部Categorical變數
# 非 Y or N 即 N

X_Categorical = ['CMFCUS1_VIP_CODE', 'CMFCUS1_BUSINESS_FLAG', 'CMFCUS1_NOTAX_FLAG', 'CMFCUS1_FINANCIAL_ACT', 
                 'CMFCUS1_AML_BUSINESS', 'DP_FG', 'LN_FG', 'IX_FG', 'BD_FG', 'FD_FG', 'WM_FG', 'TR_FG', 'EB_FG', 
                 'CD_FG', 'OT_FG', 'TRUST_YN', 'CONFIRM_YN', 'COMPLEX_CS_FG', 'AUTHORIZED', 'BEARER_SHARE', 
                 'ISSUE_BEARER', 'SOLE_CORP', 'TRUST_HOLDER', 'CUST_PANA', 'CUST_THIRD', 'CUST_ADVRS', 
                 'CUST_BAHA', 'CUST_PARA', 'CMFCUS25_AE_TYPE', 'CMFCUS25_FOREIGN_COMPANY', 'CMFCUS25_FOREIGN_ENTITY', 
                 'CMFCUS25_CERTI_FLAG', 'CMFCUS25_TAXFREE_FLAG', 'CMFCUS25_CREATIVE_FLAG', 'CMFCUS25_OSU_FLAG', 
                 'CMFCUS25_PUBLIC_CMPY']

for x in X_Categorical:
    OBU[x] = np.where(~OBU[x].isin(['Y', 'N']), 'N', OBU[x])


In [ ]:
# 將Y/N轉換為1/0以方便後面模型執行計算

# Turn 'Y', 'N' into 1, 0
for col in X_Categorical:
    OBU[col] = LabelEncoder().fit_transform(OBU[col])

##### 2.2 數值型變數

In [ ]:
# 需先經過轉換的變數
# 1. PEP_COUNT
test = OBU[(OBU['PEP_1'] == 'Y') | (OBU['PEP_2'] == 'Y') | (OBU['PEP_3'] == 'Y') | (OBU['PEP_4'] == 'Y')][['PEP_1', 'PEP_2', 'PEP_3', 'PEP_4']]
test2 = test.apply(pd.Series.value_counts, axis=1).fillna(0)
OBU['PEP_COUNT'] = 0
OBU.loc[test2.index, 'PEP_COUNT'] = test2['Y']

# 2. REL_ADVERS_COUNT
test = OBU[(OBU['REL_ADVRS_1'] == 'Y') | (OBU['REL_ADVRS_2'] == 'Y') | (OBU['REL_ADVRS_3'] == 'Y') | (OBU['REL_ADVRS_4'] == 'Y') | (OBU['REL_ADVRS_5'] == 'Y') | 
          (OBU['REL_ADVRS_6'] == 'Y') | (OBU['REL_ADVRS_7'] == 'Y') | (OBU['REL_ADVRS_8'] == 'Y') | (OBU['REL_ADVRS_9'] == 'Y') | (OBU['REL_ADVRS_10'] == 'Y') |
          (OBU['REL_ADVRS_11'] == 'Y') | (OBU['REL_ADVRS_12'] == 'Y') | (OBU['REL_ADVRS_13'] == 'Y') | (OBU['REL_ADVRS_14'] == 'Y') | (OBU['REL_ADVRS_15'] == 'Y') | 
          (OBU['REL_ADVRS_16'] == 'Y') | (OBU['REL_ADVRS_17'] == 'Y') | (OBU['REL_ADVRS_18'] == 'Y') | (OBU['REL_ADVRS_19'] == 'Y') | (OBU['REL_ADVRS_20'] == 'Y')
          ][['REL_ADVRS_1', 'REL_ADVRS_2', 'REL_ADVRS_3', 'REL_ADVRS_4', 'REL_ADVRS_5', 'REL_ADVRS_6', 'REL_ADVRS_7', 'REL_ADVRS_8', 
            'REL_ADVRS_9', 'REL_ADVRS_10', 'REL_ADVRS_11', 'REL_ADVRS_12', 'REL_ADVRS_13', 'REL_ADVRS_14', 'REL_ADVRS_15', 'REL_ADVRS_16', 
            'REL_ADVRS_17', 'REL_ADVRS_18', 'REL_ADVRS_19', 'REL_ADVRS_20']] 
test2 = test.apply(pd.Series.value_counts, axis=1).fillna(0)
OBU['REL_ADVERS_COUNT'] = 0
OBU.loc[test2.index, 'REL_ADVERS_COUNT'] = test2['Y']

# 3. REL_PEPS_COUNT
test = OBU[(OBU['REL_PEPS_1'] == 'Y') | (OBU['REL_PEPS_2'] == 'Y') | (OBU['REL_PEPS_3'] == 'Y') | (OBU['REL_PEPS_4'] == 'Y') | (OBU['REL_PEPS_5'] == 'Y') | 
          (OBU['REL_PEPS_6'] == 'Y') | (OBU['REL_PEPS_7'] == 'Y') | (OBU['REL_PEPS_8'] == 'Y') | (OBU['REL_PEPS_9'] == 'Y') | (OBU['REL_PEPS_10'] == 'Y') |
          (OBU['REL_PEPS_11'] == 'Y') | (OBU['REL_PEPS_12'] == 'Y') | (OBU['REL_PEPS_13'] == 'Y') | (OBU['REL_PEPS_14'] == 'Y') | (OBU['REL_PEPS_15'] == 'Y') | 
          (OBU['REL_PEPS_16'] == 'Y') | (OBU['REL_PEPS_17'] == 'Y') | (OBU['REL_PEPS_18'] == 'Y') | (OBU['REL_PEPS_19'] == 'Y') | (OBU['REL_PEPS_20'] == 'Y')
          ][['REL_PEPS_1', 'REL_PEPS_2', 'REL_PEPS_3', 'REL_PEPS_4', 'REL_PEPS_5', 'REL_PEPS_6', 'REL_PEPS_7', 'REL_PEPS_8', 
            'REL_PEPS_9', 'REL_PEPS_10', 'REL_PEPS_11', 'REL_PEPS_12', 'REL_PEPS_13', 'REL_PEPS_14', 'REL_PEPS_15', 'REL_PEPS_16', 
            'REL_PEPS_17', 'REL_PEPS_18', 'REL_PEPS_19', 'REL_PEPS_20']] 
test2 = test.apply(pd.Series.value_counts, axis=1).fillna(0)
OBU['REL_PEPS_COUNT'] = 0
OBU.loc[test2.index, 'REL_PEPS_COUNT'] = test2['Y']


# 全部 Numeric 變數
# 非 數字 即 0
X_Numerical = ['CMFCUS1_ADR_CNT', 'CMFCUS1_MPHONE_CNT', 'CMFCUS1_OPHONE_CNT', 
               'PEP_COUNT', 'REL_ADVERS_COUNT', 'REL_PEPS_COUNT']

for col in X_Numerical:
    OBU[col] = OBU[col].fillna(0)
    OBU[col] = OBU[col].astype(int)
    


##### 2.3 順序型變數

In [ ]:
# EY補充
# 新增欄位的情況下，將順序型變數轉換為數字順序(1, 2, 3,...)。
# 下面範例將以df代表輸入資料，NEW_COLUMN代表新增欄位
# 
# 範例一
# 新增欄位的資料內容為["L", "M", "H", Nan, ..., "H"]；欲將H/M/L轉換為3/2/1，空值(Nan)轉換為1。
# df["NEW_COLUMN"] = df["NEW_COLUMN"].map({"H": "3", "M": "2", "L": "1"}).fillna("1")


# 需先經過轉換的變數
# Remember that changing NA value into the lowest ranking in ordinal variables might cause issue and affect the model performation
# It should be adjusted to dummy variables in the second proposal.

# 1. CMFCUS1_BUSINESS_CODE
BUSINESS_CODE_TABLE = pd.read_csv(r"/home/bigdata93601/DB Result/BUSINESS_CODE_TABLE.csv", dtype=str) #行業編號對照風險等級表

# Create dictionary
BUSINESS_CODE_dict = {'L': BUSINESS_CODE_TABLE[BUSINESS_CODE_TABLE['RISK']=='L']['CODE'].tolist(),
                      'M': BUSINESS_CODE_TABLE[BUSINESS_CODE_TABLE['RISK']=='M']['CODE'].tolist(),
                      'H': BUSINESS_CODE_TABLE[BUSINESS_CODE_TABLE['RISK']=='H']['CODE'].tolist()}
# Inverse dictionary
BUSINESS_CODE_dict = {c: r for r, code in BUSINESS_CODE_dict.items() for c in code}

OBU['CMFCUS1_BUSINESS_CODE'].replace(BUSINESS_CODE_dict, inplace=True)
# Nan轉為L
OBU['CMFCUS1_BUSINESS_CODE'] = np.where(~OBU["CMFCUS1_BUSINESS_CODE"].isin(['H', 'M', 'L']), 'L', OBU['CMFCUS1_BUSINESS_CODE'])


# 2. LOCATION_CD & HEAD_OFFICE_CD 
# Create a new column: Nation_Risk to select the higher one between the two columns
NationRisk_Table = pd.read_excel(r"/home/bigdata93601/EY/NationRisk_Table.xlsx", dtype=str)

NationRisk_dict = {'1':NationRisk_Table[NationRisk_Table['洗錢風險評等']=='L']['國家代碼'].tolist(),
                   '2': NationRisk_Table[NationRisk_Table['洗錢風險評等']=='ML']['國家代碼'].tolist(),
                   '3': NationRisk_Table[NationRisk_Table['洗錢風險評等']=='M']['國家代碼'].tolist(),
                   '4': NationRisk_Table[NationRisk_Table['洗錢風險評等']=='MH']['國家代碼'].tolist(),
                   '5': NationRisk_Table[NationRisk_Table['洗錢風險評等']=='H']['國家代碼'].tolist()}

# Inverse dictionary
NationRisk_dict = {c: r for r, code in NationRisk_dict.items() for c in code}

OBU['LOCATION_CD'].replace(NationRisk_dict, inplace=True)
OBU['HEAD_OFFICE_CD'].replace(NationRisk_dict, inplace=True)
# Nan or no matching 轉為H
OBU['LOCATION_CD'] = np.where(~OBU['LOCATION_CD'].isin(['1', '2', '3', '4', '5']), '5', OBU['LOCATION_CD'])
OBU['HEAD_OFFICE_CD'] = np.where(~OBU['HEAD_OFFICE_CD'].isin(['1', '2', '3', '4', '5']), '5', OBU['HEAD_OFFICE_CD'])

OBU['NATION_RISK'] = OBU[['LOCATION_CD', 'HEAD_OFFICE_CD']].max(axis=1).astype(int).astype(str)


# 3. AMT_RANGE
# Nan 轉為0
OBU['AMT_RANGE'] = np.where(~OBU["AMT_RANGE"].isin(['1', '2', '3']), '0', OBU['AMT_RANGE'])

# 4. OBU_ANNUAL_INCOME
# Nan 轉為0
OBU['OBU_ANNUAL_INCOME'] = np.where(~OBU["OBU_ANNUAL_INCOME"].isin(['1', '2', '3']), '0', OBU['OBU_ANNUAL_INCOME'])

# 5. L, M, H to number 1, 2, 3
def risk_to_number(series):
    input = series
    if input == 'H':
        return '3'
    elif input == 'M':
        return '2'
    else:
        return '1'

for i in ['CMFCUS1_BUSINESS_CODE', 'JOB_RISK']:
    OBU[i] = OBU[i].apply(risk_to_number)

# 5. CMFCUS25_SP_RATING
OBU['CMFCUS25_SP_RATING'] = OBU['CMFCUS25_SP_RATING'].map({np.nan: 1, 'D': 2, 'SD': 3, 'R': 4, 'CCC-': 5, 'CCC': 6, 
                                                           'CCC+': 7, 'B-': 8, 'B': 9, 'B+': 10, 'BB-': 11, 'BB': 12, 
                                                           'BB+': 13, 'BBB-': 14, 'BBB': 15, 'BBB+': 16, 'A-': 17, 
                                                           'A': 18, 'A+': 19, 'AA-': 20, 'AA': 21, 'AA+': 22, 'AAA': 23})
    
# 6. CMFCUS25_MOODYS_RATING
# Moody's has assigned (P)A1 ratings to the 1 billion preferred securities shelf registration of five Travelers Capital trusts
# Here I'll take it as the same level with A1
OBU['CMFCUS25_MOODYS_RATING'] = np.where(OBU['CMFCUS25_MOODYS_RATING'] == '(P)A1', 'A1', OBU['CMFCUS25_MOODYS_RATING'])

OBU['CMFCUS25_MOODYS_RATING'] = np.where(OBU['CMFCUS25_MOODYS_RATING'] == 'A3-', 'A3', OBU['CMFCUS25_MOODYS_RATING']) #Fix typo

OBU['CMFCUS25_MOODYS_RATING'] = OBU['CMFCUS25_MOODYS_RATING'].map({np.nan: 1, 'D': 2, 'C': 3, 'Ca': 4, 'Caa3': 5, 'Caa2': 6, 
                                                                   'Caa1': 7, 'B3': 8, 'B2': 9, 'B1': 10, 'Ba3': 11, 'Ba2': 12, 
                                                                   'Ba1': 13, 'Baa3': 14, 'Baa2': 15, 'Baa1': 16, 'A3': 17, 
                                                                   'A2': 18, 'A1': 19, 'Aa3': 20, 'Aa2': 21, 'Aa1': 22, 'Aaa': 23})


# 7. CMFCUS25_SRT_SP_RATING
OBU['CMFCUS25_SRT_SP_RATING'] = OBU['CMFCUS25_SRT_SP_RATING'].map({np.nan: 1, 'D': 2, 'C': 3, 'B': 4, 'A-3': 5, 
                                                                   'A-2': 6, 'A-1': 7, 'A-1+': 8})

# 8. CMFCUS25_SRT_MOODYS_RATING
OBU['CMFCUS25_SRT_MOODYS_RATING'] = OBU['CMFCUS25_SRT_MOODYS_RATING'].map({np.nan: 1, 'P-3': 2, 'P-2': 3, 'P-1': 4})

# 9. CMFCUS25_SRT_FITCH_RATING
OBU['CMFCUS25_SRT_FITCH_RATING'] = np.where(OBU['CMFCUS25_SRT_FITCH_RATING'] == 'F2-', 'F2', OBU['CMFCUS25_SRT_FITCH_RATING']) #Fix typo

OBU['CMFCUS25_SRT_FITCH_RATING'] = OBU['CMFCUS25_SRT_FITCH_RATING'].map({np.nan: 1, 'D': 2, 'C': 3, 'B': 4, 'F3': 5, 
                                                                         'F2': 6, 'F1': 7, 'F1+': 8})


# 全部Ordinal變數
X_Ordinal = ['CMFCUS1_BUSINESS_CODE', 'NATION_RISK', 'AMT_RANGE', 
             'OBU_ANNUAL_INCOME', 'JOB_RISK', 'CMFCUS25_SP_RATING', 'CMFCUS25_MOODYS_RATING', 
             'CMFCUS25_SRT_SP_RATING', 'CMFCUS25_SRT_MOODYS_RATING', 'CMFCUS25_SRT_FITCH_RATING']

for col in X_Ordinal:
    OBU[col] = OBU[col].astype(int)

##### 2.4 啞變數(Dummy)

In [ ]:
# EY補充
# 新增欄位的情況下，確認啞變數資料內容。
# 此階段僅需確保啞變數內容正確，one hot encoding的部分會在後面執行。
# 需要注意的地方為若欲將空值作為一個單獨的類別，則建議將空值轉換為字串(EX: "NA_TYPE")。
#
# 下面範例將以df代表輸入資料，NEW_COLUMN代表新增欄位
#
# 範例一
# 新增欄位的資料內容為["A", "C", "1", "3", Nan,..., Nan]；欲將空值(Nan)轉換為"NA_TYPE"，其餘內容保持不變。
# df["NEW_COLUMN"] = np.where(df["NEW_COLUMN"].isna(), "NA_TYPE", df["NEW_COLUMN"])
#
# 範例二
# 新增欄位的資料內容為["A", "C", "B", "DELAY", "Q", Nan, ..., "A"]；欲將A, B, C, Q保留，其餘內容分類為空值類別。
# df["NEW_COLUMN"] = np.where(df["NEW_COLUMN"].isin(["A", "B", "C", "Q"]), df["NEW_COLUMN"], "NA_TYPE")


# 需先經過轉換的變數
# 1. CMFCUS1_BIRTH_DATE
OBU['CMFCUS1_BIRTH_DATE'] = pd.to_datetime(OBU['CMFCUS1_BIRTH_DATE'], errors='coerce') #if out of bounds then return NAT

def calculate_age(born):
    today = date.today()
    return today.year - born.year - ((today.month, today.day) < (born.month, born.day))

OBU['CMFCUS1_BIRTH_DATE'] = OBU['CMFCUS1_BIRTH_DATE'].apply(calculate_age)

def apply_age(Series):
    input = Series
    if input < 3:
        return '0-3'
    elif (input >= 3) & (input < 5):
        return '3-5'
    elif (input >= 5) & (input < 10):
        return '5-10'
    elif input >= 10:
        return 'larger_than_10'
    else:
        'NA_TYPE'

OBU['CMFCUS1_BIRTH_DATE'] = OBU['CMFCUS1_BIRTH_DATE'].apply(apply_age)


# 2. CMFCUS1_Q_ID
# 00, 01, 03, 10, 36, 49, 99都歸於NA_TYPE
OBU['CMFCUS1_Q_ID'] = np.where(OBU["CMFCUS1_Q_ID"].isin(['00', '01', '03', '10', '36', '49', '99', 'ZZ', np.nan]), 
                               'NA_TYPE', OBU["CMFCUS1_Q_ID"])

# 3. 'CMFCUS1_PURPOSE', 'CMFCUS1_DERIVATIVE', 'CMFCUS1_TITLE_CODE', 'LN_TYP', 'CONFIRM_TYPE', 'CMFCUS25_BRANCH'
# NA改成NA_TYPE
for i  in ['CMFCUS1_JOB_TITLE', 'CMFCUS1_PURPOSE', 'CMFCUS1_DERIVATIVE', 'CMFCUS1_TITLE_CODE', 'LN_TYP', 'CONFIRM_TYPE', 'CMFCUS25_BRANCH', 'CMFCUS25_FORE_CASH_FLAG']:
    OBU[i] =np.where(OBU[i].isna(), 'NA_TYPE', OBU[i])

# 4. CORP_TYPE
# 根據IssueLog Mega回覆: 共12類
# 1~9, A, Z, Nan: NA_TYPE

OBU['CORP_TYPE'] = np.where(OBU['CORP_TYPE'].isin(['1', '2', '3', '4', '5', '6', '7', '8', '9', 'Z', 'A']), OBU['CORP_TYPE'], 'NA_TYPE')


# 全部Dummy變數
X_Dummy = ['CMFCUS1_BIRTH_DATE', 'CMFCUS1_JOB_TITLE', 'CMFCUS1_Q_ID','CMFCUS1_PURPOSE', 'CMFCUS1_DERIVATIVE', 'CMFCUS1_TITLE_CODE', 
           'CUST_TYP', 'LN_TYP', 'CONFIRM_TYPE', 'CORP_TYPE', 'CMFCUS25_BRANCH', 'CMFCUS25_FORE_CASH_FLAG', 'CHANNEL']


#### 3. 模型建立

In [ ]:
OBU['SAR'] = LabelEncoder().fit_transform(OBU['SAR'])
X_col = X_Categorical + X_Numerical + X_Ordinal + X_Dummy

X = OBU[X_col] 

# 轉換dummy變數
for each in X_Dummy:
    dummies = pd.get_dummies(X.loc[:, each], prefix=each)
    X = pd.concat([X, dummies], axis=1)
    X = X.drop(each, axis=1) # delete original Dummy column

y = OBU['SAR'] 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, stratify=y) #stratify確保訓練資料SAR比例一致

##### 3.1 Parameter Tuning
Input the parameters we would like to use grid search

In [ ]:
# EY補充
# 下面程式碼為選擇需要進行Tuning的參數
# 詳細模型可使用參數請參考官方網頁: https://xgboost.readthedocs.io/en/stable/parameter.html


# 參數調整
eval_metric = ['error', 'logloss', 'aucpr']

space={# parameters we would like to tune
       'eta': hp.uniform('eta', 0.2, 0.6),
       'max_depth': hp.quniform('max_depth', 2, 6, 1), #調整樹的深度
       'min_child_weight': hp.quniform('min_child_weight', 0, 15, 1), #調整子集合權重
       'max_delta_step': hp.quniform('max_delta_step', 0, 10, 1),
       'subsample': hp.uniform('subsample', 0.5, 1), #調整重覆抽樣比例
       'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1),
       'gamma': hp.quniform('gamma', 0, 20, 1),
       'reg_alpha': hp.quniform('reg_alpha', 20, 180, 1), #alpha值設定
       'reg_lambda': hp.uniform('reg_lambda', 0, 1), #beta值設定
       'scale_pos_weight': hp.quniform('scale_pos_weight', 50, 120, 1), # 應變數權重調整NO_SAR : SAR = 1:75
       'eval_metric': hp.choice('eval_metric', eval_metric), 
       'n_estimators': hp.quniform('n_estimators', 100, 200, 10) 
       }

Define Objective Function

In [ ]:
# EY補充
# 下面程式碼為定義參數Tuning之函式

def objective(space):
    model = xgb.XGBClassifier(eta=space['eta'],
                              max_depth=int(space['max_depth']), 
                              min_child_weight=int(space['min_child_weight']), 
                              max_delta_step=int(space['max_delta_step']),
                              subsample=space['subsample'], 
                              colsample_bytree=space['colsample_bytree'],
                              gamma=space['gamma'],
                              reg_alpha=int(space['reg_alpha']),
                              reg_lambda=space['reg_lambda'],
                              scale_pos_weight=int(space['scale_pos_weight']),
                              eval_metric=space['eval_metric'], # aucpr, logloss, error
                              n_estimators=int(space['n_estimators']),
                              
                              # static parameters
                              objective='binary:logistic',
                              random_state=1) 
    
    evaluation = [(X_train, y_train), (X_test, y_test)]
    
    model.fit(X_train, y_train, eval_set=evaluation, early_stopping_rounds=30, verbose=False) # verbose=False would prevent printing each iteration outcome
    y_pred = model.predict(X_test)
    
    # EY補充
    # 下面recall score, roc, accuracy & f1 score 為定義模型好壞之指標(loss function)
    # 此處詳列多個不同指標，實際進行參數Tuning時僅需選擇其一
    recall_score = sklearn.metrics.recall_score(y_test, y_pred)
    roc_auc_score = sklearn.metrics.roc_auc_score(y_test, y_pred)
    accuracy = sklearn.metrics.accuracy_score(y_test, y_pred > 0.5)
    f1_score = sklearn.metrics.f1_score(y_test, y_pred)
    
    # print(sklearn.metrics.classification_report(y_test, y_pred))
    # print("SCORE: ", recall_score)
    # for the loss function: 
    # accuracy: -accuracy
    # recall: 1-recall
    # precision: 1-precision
    # roc_auc_score: 1-roc-auc_score
    
    # EY補充
    # 下面程式碼之目的為選擇定義模型好壞之指標(loss function)
    # 使用者也可以透過相乘的方式使用複合性指標
    return {'loss': (1-recall_score) * (1-accuracy),  # (1-recall_score) * (1-accuracy); 1-recall_score, -accuracy, -f1_score
            'status': hyperopt.STATUS_OK} 

In [ ]:
# 調參：計算最適合的參數
trials = hyperopt.Trials()

# EY補充
# 下面程式碼為實際執行參數Tuning
# max_evals為設定迭代之次數，須注意迭代次數愈大，程式碼執行時間愈久
best_hyperparams = hyperopt.fmin(fn = objective, 
                                 space = space, 
                                 algo = hyperopt.tpe.suggest, 
                                 max_evals=100, # iteration number
                                 trials = trials)

In [ ]:
# 查看最佳解之參數結果
print("The best hyperparameters are : ", '\n')
print(best_hyperparams)

##### 3.2 Cross-Validation

In [ ]:
# 執行模型進行交叉驗證
model = xgb.XGBClassifier(booster='gbtree', 
                          
                          # Model Parameters
                          n_estimators=int(best_hyperparams['n_estimators']), # The number of rounds for boosting
                          eta=best_hyperparams['eta'],
                          max_depth=int(best_hyperparams['max_depth']), # default=6
                          min_child_weight=int(best_hyperparams['min_child_weight']), # default=1; The larger the min_child_weight is, the more conservative the algorithm will be
                          max_delta_step=int(best_hyperparams['max_delta_step']),
                          subsample=best_hyperparams['subsample'], # default=1; subsample ratio of the training instances.
                          colsample_bytree=best_hyperparams['colsample_bytree'], # default=1, how many columns used in each tree training
                          scale_pos_weight=int(best_hyperparams['scale_pos_weight']), # adjust y value weights
                          
                          # Learning Parameters
                          objective='binary:logistic',
                          gamma=best_hyperparams['gamma'], # default=0; the larger gamma is, the more conservative the algorithm will be
                          reg_lambda=best_hyperparams['reg_lambda'], # default=1; L2 regularization term on weights. Increasing this value will make model more conservative
                          reg_alpha=int(best_hyperparams['reg_alpha']), # default=1; L1 regularization
                          random_state=1, 
                          eval_metric=eval_metric[best_hyperparams['eval_metric']])

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=10)
print('Recall: ', cross_val_score(model, X, y, cv=skf, scoring='recall')) # default scoring='accuracy'
print('Accuracy: ', cross_val_score(model, X, y, cv=skf, scoring='accuracy')) # default scoring='accuracy'

##### 3.3 模型指標確認

In [ ]:
# 執行模型並進行交叉驗證並檢驗AUC之穩定度
model = xgb.XGBClassifier(booster='gbtree', 
                          
                          # Model Parameters
                          n_estimators=int(best_hyperparams['n_estimators']), # The number of rounds for boosting
                          eta=best_hyperparams['eta'],
                          max_depth=int(best_hyperparams['max_depth']), # default=6
                          min_child_weight=int(best_hyperparams['min_child_weight']), # default=1; The larger the min_child_weight is, the more conservative the algorithm will be
                          max_delta_step=int(best_hyperparams['max_delta_step']),
                          subsample=best_hyperparams['subsample'], # default=1; subsample ratio of the training instances.
                          colsample_bytree=best_hyperparams['colsample_bytree'], # default=1, how many columns used in each tree training
                          scale_pos_weight=int(best_hyperparams['scale_pos_weight']), # adjust y value weights
                          
                          # Learning Parameters
                          objective='binary:logistic',
                          gamma=best_hyperparams['gamma'], # default=0; the larger gamma is, the more conservative the algorithm will be
                          reg_lambda=best_hyperparams['reg_lambda'], # default=1; L2 regularization term on weights. Increasing this value will make model more conservative
                          reg_alpha=int(best_hyperparams['reg_alpha']), # default=1; L1 regularization
                          random_state=1, 
                          eval_metric=eval_metric[best_hyperparams['eval_metric']])

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=10)
print('Recall: ', cross_val_score(model, X, y, cv=skf, scoring='recall')) # default scoring='accuracy'
print('Accuracy: ', cross_val_score(model, X, y, cv=skf, scoring='accuracy')) # default scoring='accuracy'
print('------------------------------------------------------------')
# 檢查Confidence Interval on AUC
auc_scores = cross_val_score(model, X, y, cv=skf, scoring='roc_auc')
print(auc_scores)
print("AUC Confidence Interval: %0.2f (+/- %0.2f)" % (auc_scores.mean(), auc_scores.std() * 1.96))

In [ ]:
# 模型結果檢視(Presentation)
OBU['FINAL_CRR'] = pd.Categorical(OBU['FINAL_CRR'], ['H', 'M', 'L'])

for train_index, test_index in skf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    model.fit(X_train, y_train)
    
    # Record Predict Outcome
    OBU.loc[test_index, 'pred'] = model.predict(X_test)
    # Record Probability
    OBU.loc[test_index, 'pred_proba'] = model.predict_proba(X_test)[:, 1].astype(float)
    print(OBU.iloc[test_index].groupby(['FINAL_CRR', 'SAR'])['pred_proba'].mean().reset_index().sort_values(by=['FINAL_CRR', 'SAR'], ascending=[True, False]))

In [ ]:
# 模型結果檢視(Presentation)
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    model.fit(X_train, y_train)
    
    # Record Predict Outcome
    OBU.loc[test_index, 'pred'] =model.predict(X_test)
    # Record Probability
    OBU.loc[test_index, 'pred_proba'] = model.predict_proba(X_test)[:, 1].astype(float)
    print(OBU.iloc[test_index].groupby(['FINAL_CRR'])['pred_proba'].mean())

In [ ]:
# 模型指標檢視：Accuracy, Recall, AUC
confusion_matrix = pd.crosstab(OBU['SAR'], OBU['pred'], rownames=['Actual'], 
                              colnames=['Predicted'])
sn.heatmap(confusion_matrix, annot=True)
print('Accuracy: ', metrics.accuracy_score(OBU['SAR'], OBU['pred']))
plt.show()

recall_score = metrics.recall_score(OBU['SAR'], OBU['pred'], pos_label=1)
precision_score = metrics.precision_score(OBU['SAR'], OBU['pred'], pos_label=1)
print('Recall Score: ', recall_score)
print('Precision Score: ', precision_score)


print('Area Under Curve: ', roc_auc_score(OBU['SAR'], OBU['pred_proba']))

fpr, tpr, threshold = metrics.roc_curve(OBU['SAR'], OBU['pred_proba'], pos_label=1)
roc_auc = metrics.auc(fpr, tpr)

plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label='ROC Curve (AUC = %0.2f)' % roc_auc)
plt.legend(loc='lower right')
plt.plot([0,1], [0,1], 'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
# 模型指標檢視(Presentation)
print(sklearn.metrics.classification_report(OBU['SAR'], OBU['pred']))

##### 3.4 XGBoost Variable Importance

In [ ]:
# XGBoost 重要欄位檢視；僅代表欄位被選擇之次數，不代表指向性(正負)
'''
"weight" is the number of times a features appears in a tree
"gain" is the average gain of splits which use the feature
"cover" is the average coverage of splits which use the feature where coverage is defined as the number of samples affected by the split
'''


xgb.plot_importance(model, max_num_features=15, importance_type='gain') # default: importance_type='weight'
plt.show()

In [ ]:
# XGBoost 重要欄位檢視(前15名)；僅代表欄位被選擇之次數，不代表指向性(正負)
im = pd.DataFrame({'Column': X.columns, 'Importance': model.feature_importances_})
im = im.sort_values(by='Importance', ascending=False)
im.head(15)

#### Appendix 1

In [ ]:
# 查看SAR分佈
OBU['SAR'].value_counts(ascending=True)

In [ ]:
# FINAL_CRR & SAR的數量分佈
OBU.groupby(['FINAL_CRR', 'SAR'])['CUST_ID'].count().reset_index().sort_values(by=['FINAL_CRR', 'SAR'], ascending=[True, False])

In [ ]:
# 查看FINAL_CRR分佈
OBU.groupby(['FINAL_CRR'])['CUST_ID'].count()

#### Appendix 2
輸出機率數值供切分點分析  
計算當機率切分為40%~60%時，F1-Score or Recall Score 之最佳化

In [ ]:
test = OBU.loc[:, ['pred_proba', 'FINAL_CRR', 'SAR']]

In [ ]:
for i in range(40, 66, 5):
    test['pred'] = np.where(test['pred_proba'] >= i/100, 1, 0)
    print('機率切分為', i, '%:')
    print('Recall Score: ', metrics.recall_score(test['SAR'], test['pred']))
    print('Accuracy: ', metrics.accuracy_score(test['SAR'], test['pred']))
    print('F1 Score: ', metrics.f1_score(test['SAR'], test['pred']))
    print('-------------------------------------------')